# Mason's workbook on zestimate log errors

##### Due: 2022.01.07

In [1]:
#learn gong fu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mason_functions as mf
import scaler as sc
import wrangle_bits as wb
import sklearn.preprocessing

import env
import os
import warnings
warnings.filterwarnings('ignore')

from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.feature_selection import SelectKBest, f_regression, RFE

## Plan

## Wrangle

In [2]:
#define my sql query
sql = '''
SELECT *
FROM properties_2017
LEFT JOIN predictions_2017 pred USING(parcelid)
LEFT JOIN airconditioningtype USING(airconditioningtypeid)
LEFT JOIN architecturalstyletype USING(architecturalstyletypeid)
LEFT JOIN buildingclasstype USING(buildingclasstypeid)
LEFT JOIN heatingorsystemtype USING(heatingorsystemtypeid)
LEFT JOIN propertylandusetype USING(propertylandusetypeid)
LEFT JOIN storytype USING(storytypeid)
LEFT JOIN typeconstructiontype USING(typeconstructiontypeid)
WHERE latitude IS NOT NULL 
AND longitude IS NOT NULL
AND propertylandusetypeid = 261
AND transactiondate LIKE "2017%%"
AND pred.id IN (SELECT MAX(id)
FROM predictions_2017
GROUP BY parcelid
HAVING MAX(transactiondate))
'''

#define my url
url = mf.get_db_url('zillow')

#set up an if-conditional to see if there is a .csv readily available, then load the data
if os.path.isfile('zillow_ol_pal.csv'):
    df = pd.read_csv('zillow_ol_pal.csv', index_col = 0)
else:
    df = pd.read_sql(sql, url)
    df.to_csv('zillow_ol_pal.csv')

In [3]:
pd.options.display.max_columns = 69
pd.options.display.max_rows = 69

In [4]:
df.head()

,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,parcelid,id,basementsqft,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id.1,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14297519,1727539,NaN,3.5,4.0,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,NaN,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,1.0,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13,0,0.025595,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,17052889,1387261,NaN,1.0,2.0,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,NaN,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13,1,0.055619,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,14186244,11677,NaN,2.0,3.0,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,NaN,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13,2,0.005383,2017-01-01,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN
3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,12177905,2288172,NaN,3.0,4.0,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,1.0,0101,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13,3,-0.103410,2017-01-01,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN
4,NaN,NaN,261.0,2.0,NaN,NaN,1.0,12095076,781532,NaN,3.0,4.0,9.0,3.0,NaN,NaN,2962.0,2962.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,34145202.0,-118179824.0,63000.0,1.0,NaN,NaN,NaN,1.0,0101,PSR2,6.037461e+07,47019.0,3101.0,274684.0,96293.0,0.0,NaN,1.0,NaN,NaN,1950.0,NaN,NaN,276684.0,773303.0,2016.0,496619.0,9516.26,NaN,NaN,6.037461e+13,6,-0.001011,2017-01-01,Central,NaN,NaN,Central,Single Family Residential,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52319 entries, 0 to 52318
Data columns (total 69 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        76 non-null     float64
 1   storytypeid                   47 non-null     float64
 2   propertylandusetypeid         52319 non-null  float64
 3   heatingorsystemtypeid         33849 non-null  float64
 4   buildingclasstypeid           0 non-null      float64
 5   architecturalstyletypeid      70 non-null     float64
 6   airconditioningtypeid         13615 non-null  float64
 7   parcelid                      52319 non-null  int64  
 8   id                            52319 non-null  int64  
 9   basementsqft                  47 non-null     float64
 10  bathroomcnt                   52319 non-null  float64
 11  bedroomcnt                    52319 non-null  float64
 12  buildingqualitytypeid         33654 non-null  float64
 13  c

In [6]:
#verify length of dataframe is the same as the number of unique parcel ids
df.parcelid.nunique()

52319

In [7]:
#calculate column nulls
column_nulls = wb.calculate_column_nulls(df)

In [8]:
#column nulls at a glance
column_nulls.sort_values('nulls', ascending = False)

,nulls,null_ratio,null_percentage
finishedsquarefeet13,52319,1.00000,100.0%
buildingclassdesc,52319,1.00000,100.0%
buildingclasstypeid,52319,1.00000,100.0%
finishedsquarefeet15,52319,1.00000,100.0%
storydesc,52272,0.99910,99.91%
storytypeid,52272,0.99910,99.91%
basementsqft,52272,0.99910,99.91%
yardbuildingsqft26,52256,0.99880,99.88%
architecturalstyletypeid,52249,0.99866,99.87%
architecturalstyledesc,52249,0.99866,99.87%


In [9]:
#calculate row nulls
row_nulls = wb.calculate_row_nulls(df)

In [10]:
#row nulls at a glance
row_nulls.sort_values('nulls', ascending = False)

,nulls,null_ratio,null_percentage
28006,48,0.69565,70%
8954,48,0.69565,70%
41347,47,0.68116,68%
13356,46,0.66667,67%
21989,45,0.65217,65%
...,...,...,...
44684,24,0.34783,35%
83,24,0.34783,35%
37444,24,0.34783,35%
39391,23,0.33333,33%


In [11]:
#handle missing nulls 
#(drop any column with over half of the values missing and any rows with over half missing after dropping columns)
df = wb.handle_nulls(df, .5 , .5)

In [12]:
#calculate remaining column nulls
column_nulls = wb.calculate_column_nulls(df)

In [13]:
#remaining column nulls at a glance
column_nulls.sort_values('nulls', ascending = False)

,nulls,null_ratio,null_percentage
buildingqualitytypeid,18665,0.35675,35.68%
unitcnt,18558,0.35471,35.47%
propertyzoningdesc,18558,0.35471,35.47%
heatingorsystemdesc,18470,0.35303,35.3%
heatingorsystemtypeid,18470,0.35303,35.3%
regionidcity,1036,0.01980,1.98%
lotsizesquarefeet,366,0.00700,0.7%
finishedsquarefeet12,246,0.00470,0.47%
fullbathcnt,135,0.00258,0.26%
calculatedbathnbr,135,0.00258,0.26%


In [14]:
#calculate remaining row nulls
row_nulls = wb.calculate_row_nulls(df)

In [15]:
#remaining row nulls at a glance
row_nulls.sort_values('nulls', ascending = False)

,nulls,null_ratio,null_percentage
28006,14,0.40000,40%
8954,14,0.40000,40%
41347,13,0.37143,37%
48475,12,0.34286,34%
32311,12,0.34286,34%
...,...,...,...
21482,0,0.00000,0%
21483,0,0.00000,0%
21484,0,0.00000,0%
21486,0,0.00000,0%


In [16]:
#set an empty list
okay_to_drop = []

#commence loop through index of column nulls
for n in range(len(column_nulls)):
    
    #set up conditional to see if ratio of values missing in column is less than 2%
    if column_nulls.iloc[n].null_ratio < .2:
        
        #add features to list 
        okay_to_drop.append(column_nulls.index[n])
        
#verify
#print(okay_to_drop)

#commence loop through list        
for feature in okay_to_drop:
    
    #drop rows where there are missing values in the columns specified
    df = df[df[feature].notnull()]

In [17]:
#calculate remaining nulls
column_nulls = wb.calculate_column_nulls(df)

In [18]:
#pull up the list of column nulls to verify dropped rows
column_nulls.sort_values('nulls', ascending = False)

,nulls,null_ratio,null_percentage
buildingqualitytypeid,17939,0.35481,35.48%
propertyzoningdesc,17873,0.35351,35.35%
unitcnt,17856,0.35317,35.32%
heatingorsystemdesc,17764,0.35135,35.14%
heatingorsystemtypeid,17764,0.35135,35.14%
propertylandusetypeid,0,0.00000,0.0%
taxvaluedollarcnt,0,0.00000,0.0%
roomcnt,0,0.00000,0.0%
yearbuilt,0,0.00000,0.0%
structuretaxvaluedollarcnt,0,0.00000,0.0%


What is propertyzoningdesc? There is no corresponding table

In [19]:
#pull up values for propertyzoningdesc column
df.propertyzoningdesc.value_counts(dropna = False)

NaN           17873
LAR1           6404
LARS           1378
LBR1N          1107
SCUR2           611
              ...  
LCR 2             1
AHRPD51U-R        1
PDR1-13000        1
TCR4YY            1
TOCC-GEN          1
Name: propertyzoningdesc, Length: 1237, dtype: int64

In [20]:
#create final list of columns to drop
final_drops = []

In [21]:
#add column to list
final_drops.append('propertyzoningdesc')

In [22]:
#pull up values for heatingsystemdesc
df.heatingorsystemdesc.value_counts(dropna = False)

Central       19897
NaN           17764
Floor/Wall    12330
Forced air      502
Solar            36
None             16
Baseboard         7
Radiant           4
Gravity           2
Yes               1
Name: heatingorsystemdesc, dtype: int64

Okay, I'll come back for this

In [23]:
#pull up values for unitcnt
df.unitcnt.value_counts(dropna = False)

1.0    32684
NaN    17856
2.0       18
3.0        1
Name: unitcnt, dtype: int64

In [24]:
#map null values to the value of 1
df.unitcnt = df.unitcnt.map({np.nan: 1,
               1: 1,
               2: 2,
               3: 3})

In [25]:
#verify
df.unitcnt.value_counts(dropna = False)

1    50540
2       18
3        1
Name: unitcnt, dtype: int64

In [26]:
#pull up values for buildingqualitytypeid
df.buildingqualitytypeid.value_counts(dropna = False).sort_index(ascending = False)

12.0       76
11.0      482
10.0      211
9.0      1035
8.0      7981
7.0      3125
6.0     10125
5.0      1469
4.0      7967
3.0       141
2.0         4
1.0         4
NaN     17939
Name: buildingqualitytypeid, dtype: int64

There is nothing in the database that I see that helps me identify what kind of feature this is. Drop

In [27]:
#add to the list of final drops
final_drops.append('buildingqualitytypeid')

In [28]:
#drop columns on list
df = df.drop(columns = final_drops)

In [29]:
#calculate remaining column nulls
column_nulls = wb.calculate_column_nulls(df)

In [30]:
#pull up list of sorted column nulls
column_nulls.sort_values('nulls', ascending = False)

,nulls,null_ratio,null_percentage
heatingorsystemtypeid,17764,0.35135,35.14%
heatingorsystemdesc,17764,0.35135,35.14%
assessmentyear,0,0.00000,0.0%
regionidzip,0,0.00000,0.0%
roomcnt,0,0.00000,0.0%
unitcnt,0,0.00000,0.0%
yearbuilt,0,0.00000,0.0%
structuretaxvaluedollarcnt,0,0.00000,0.0%
taxvaluedollarcnt,0,0.00000,0.0%
propertylandusetypeid,0,0.00000,0.0%


In [31]:
#age as a continuous variable makes more sense
df['age'] = 2017 - df['yearbuilt']
df = df.drop(columns = 'yearbuilt')

In [32]:
#get a list of numerical columns 
quant_vars = ['bathroomcnt', 'bedroomcnt', 'calculatedbathnbr', 'unitcnt', 'calculatedfinishedsquarefeet', 'fullbathcnt', 'lotsizesquarefeet', 'roomcnt', 'structuretaxvaluedollarcnt', 'taxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxamount', 'age']

In [33]:
#get info on numeric columns
df[quant_vars].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50559 entries, 0 to 52318
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   bathroomcnt                   50559 non-null  float64
 1   bedroomcnt                    50559 non-null  float64
 2   calculatedbathnbr             50559 non-null  float64
 3   unitcnt                       50559 non-null  int64  
 4   calculatedfinishedsquarefeet  50559 non-null  float64
 5   fullbathcnt                   50559 non-null  float64
 6   lotsizesquarefeet             50559 non-null  float64
 7   roomcnt                       50559 non-null  float64
 8   structuretaxvaluedollarcnt    50559 non-null  float64
 9   taxvaluedollarcnt             50559 non-null  float64
 10  landtaxvaluedollarcnt         50559 non-null  float64
 11  taxamount                     50559 non-null  float64
 12  age                           50559 non-null  float64
dtypes

In [34]:
#set parcel id to index
df = df.set_index('parcelid')

In [35]:
#assign dataframe columns to a list
df_columns = df.columns.to_list()

In [36]:
#loop through the list of columns
for col in quant_vars:

    q1, q3 = df[col].quantile([.25, .75])  # get quartiles
    
    iqr = q3 - q1   # calculate interquartile range
    
    upper_bound = q3 + 1 * iqr   # get upper bound
    lower_bound = q1 - 1 * iqr   # get lower bound
    
    dft = df[(df[col] > lower_bound) & (df[col] < upper_bound)] # return dataframe without outliers

In [37]:
#verify some entries were removed (pull of dataframe info)
dft.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47235 entries, 14297519 to 12826780
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   propertylandusetypeid         47235 non-null  float64
 1   heatingorsystemtypeid         30488 non-null  float64
 2   id                            47235 non-null  int64  
 3   bathroomcnt                   47235 non-null  float64
 4   bedroomcnt                    47235 non-null  float64
 5   calculatedbathnbr             47235 non-null  float64
 6   calculatedfinishedsquarefeet  47235 non-null  float64
 7   finishedsquarefeet12          47235 non-null  float64
 8   fips                          47235 non-null  float64
 9   fullbathcnt                   47235 non-null  float64
 10  latitude                      47235 non-null  float64
 11  longitude                     47235 non-null  float64
 12  lotsizesquarefeet             47235 non-null  floa

In [38]:
df = dft

In [39]:
fill_these = df[df.heatingorsystemdesc.isnull()]
fill_these.head()

,propertylandusetypeid,heatingorsystemtypeid,id,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,latitude,longitude,lotsizesquarefeet,propertycountylandusecode,rawcensustractandblock,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id.1,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc,age
parcelid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
14297519,261.0,NaN,1727539,3.5,4.0,3.5,3100.0,3100.0,6059.0,3.0,33634931.0,-117869207.0,4506.0,122,6.059063e+07,53571.0,1286.0,96978.0,0.0,1,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,0,0.025595,2017-01-01,NaN,Single Family Residential,19.0
17052889,261.0,NaN,1387261,1.0,2.0,1.0,1465.0,1465.0,6111.0,1.0,34449266.0,-119281531.0,12647.0,1110,6.111001e+07,13091.0,2061.0,97099.0,5.0,1,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13,1,0.055619,2017-01-01,NaN,Single Family Residential,50.0
14186244,261.0,NaN,11677,2.0,3.0,2.0,1243.0,1243.0,6059.0,2.0,33886168.0,-117823170.0,8432.0,122,6.059022e+07,21412.0,1286.0,97078.0,6.0,1,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,2,0.005383,2017-01-01,NaN,Single Family Residential,55.0
13944538,261.0,NaN,249412,2.5,3.0,2.5,1340.0,1340.0,6059.0,2.0,33771438.0,-117982834.0,1199.0,122,6.059089e+07,24832.0,1286.0,97051.0,6.0,1,96617.0,319668.0,2016.0,223051.0,4078.08,6.059089e+13,13,0.045602,2017-01-02,NaN,Single Family Residential,37.0
17110996,261.0,NaN,43675,2.5,3.0,2.5,1371.0,1371.0,6111.0,2.0,34238229.0,-119170633.0,3445.0,1111,6.111005e+07,13150.0,2061.0,97107.0,5.0,1,99026.0,198054.0,2016.0,99028.0,2204.84,6.111005e+13,15,0.008669,2017-01-02,NaN,Single Family Residential,13.0


In [40]:
df = df[df.heatingorsystemtypeid != 24]

In [41]:
ClaX_train = train

NameError: name 'train' is not defined

In [42]:
#a look at the value counts 
df.rawcensustractandblock.value_counts().sort_index(ascending = False)

6.111009e+07    1
6.111009e+07    1
6.111009e+07    1
6.111009e+07    1
6.111009e+07    1
               ..
6.037101e+07    2
6.037101e+07    1
6.037101e+07    1
6.037101e+07    1
6.037101e+07    1
Name: rawcensustractandblock, Length: 28652, dtype: int64

In [43]:
df.censustractandblock.value_counts().sort_values(ascending = True)

6.059100e+13     1
6.037301e+13     1
6.037535e+13     1
6.111004e+13     1
6.037552e+13     1
                ..
6.037194e+13    20
6.037142e+13    21
6.059032e+13    21
6.059032e+13    24
6.037920e+13    32
Name: censustractandblock, Length: 28650, dtype: int64

In [44]:
#assign new column to object-values (did not work) for the raw census tract and block numbers
df['tractsense'] = df['rawcensustractandblock'].astype('object')

In [45]:
#pull up value counts 
df.tractsense.value_counts()

6.037920e+07    32
6.059032e+07    24
6.059032e+07    21
6.037142e+07    21
6.037194e+07    20
                ..
6.059002e+07     1
6.037555e+07     1
6.059063e+07     1
6.059111e+07     1
6.037574e+07     1
Name: tractsense, Length: 28652, dtype: int64

In [46]:
#raw census data sorted (what is this feature, really?)
df.tractsense.sort_values(ascending = False)

parcelid
17162024    61110091.001017
17162450    61110091.001004
17162416    61110091.001002
17169061    61110089.002025
17167911    61110089.002022
                 ...       
11001108    60371011.101011
11002186    60371011.101009
11002100    60371011.101002
11002060    60371011.101001
11002019       60371011.101
Name: tractsense, Length: 47234, dtype: object

In [47]:
#add these columns to the list of columns to drop
final_drops.append('censustractandblock')
final_drops.append('rawcensustractandblock')
final_drops.append('tractsense')
final_drops.append('finishedsquarefeet12')
final_drops.append('regionidcity')
final_drops.append('regionidcounty')
final_drops.append('id.1')
                   
#remove these from the list, cannot drop again
final_drops.remove('buildingqualitytypeid')
final_drops.remove('propertyzoningdesc')

In [48]:
final_drops

['censustractandblock',
 'rawcensustractandblock',
 'tractsense',
 'finishedsquarefeet12',
 'regionidcity',
 'regionidcounty',
 'id.1']

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47234 entries, 14297519 to 12826780
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   propertylandusetypeid         47234 non-null  float64
 1   heatingorsystemtypeid         30487 non-null  float64
 2   id                            47234 non-null  int64  
 3   bathroomcnt                   47234 non-null  float64
 4   bedroomcnt                    47234 non-null  float64
 5   calculatedbathnbr             47234 non-null  float64
 6   calculatedfinishedsquarefeet  47234 non-null  float64
 7   finishedsquarefeet12          47234 non-null  float64
 8   fips                          47234 non-null  float64
 9   fullbathcnt                   47234 non-null  float64
 10  latitude                      47234 non-null  float64
 11  longitude                     47234 non-null  float64
 12  lotsizesquarefeet             47234 non-null  floa

In [50]:
df.id.value_counts(ascending = False)

749775     1
1833738    1
34703      1
1293058    1
3126       1
          ..
490789     1
2581798    1
1469735    1
2168104    1
657407     1
Name: id, Length: 47234, dtype: int64

In [51]:
df.heatingorsystemtypeid = df.heatingorsystemdesc.astype(str)

In [54]:
#convert id to object type
df.id = df.id.astype(str)
df.propertylandusetypeid = df.propertylandusetypeid.astype(str)
df.tractsense = df.tractsense.astype(str)
df.assessmentyear = df.assessmentyear.astype(str)

#reduce noice by chainging columns values to integers instead of decimals
df.fips = df.fips.astype(int)
df.regionidzip = df.regionidzip.astype(int)
df.age = df.age.astype(int)
df.lotsizesquarefeet = df.lotsizesquarefeet.astype(int)
df.calculatedfinishedsquarefeet = df.calculatedfinishedsquarefeet.astype(int)

In [55]:
#at a glance 
df.id.sort_values(ascending = False)

parcelid
17167083     999982
17157684     999939
17144392     999884
17129835     999816
17121156     999777
             ...   
17225734    1000194
17225311    1000191
13902719     100019
17206110    1000112
17193708    1000072
Name: id, Length: 47234, dtype: object

In [56]:
#pull up value coutns for city region id 
df.regionidcity.value_counts()

12447.0    10004
5534.0      1559
40227.0     1335
46298.0     1300
16764.0     1046
           ...  
13311.0        3
31134.0        2
36078.0        1
10815.0        1
21395.0        1
Name: regionidcity, Length: 175, dtype: int64

In [57]:
df.regionidcounty.value_counts()

3101.0    30216
1286.0    12902
2061.0     4116
Name: regionidcounty, dtype: int64

In [58]:
df.regionidzip.value_counts()

97319    653
97318    570
97329    506
97328    501
97118    471
        ... 
95996      1
96289      1
96148      1
96467      1
95987      1
Name: regionidzip, Length: 374, dtype: int64

In [59]:
len(df[df.fips == 6037])

30216

In [60]:
len(df[df.fips == 6111])

4116

In [61]:
final_drops

['censustractandblock',
 'rawcensustractandblock',
 'tractsense',
 'finishedsquarefeet12',
 'regionidcity',
 'regionidcounty',
 'id.1']

In [62]:
#drop the columns deemed superfluous
df = df.drop(columns = final_drops)

In [63]:
#verify changes to df (pull up dataframe info)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47234 entries, 14297519 to 12826780
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   propertylandusetypeid         47234 non-null  object 
 1   heatingorsystemtypeid         47234 non-null  object 
 2   id                            47234 non-null  object 
 3   bathroomcnt                   47234 non-null  float64
 4   bedroomcnt                    47234 non-null  float64
 5   calculatedbathnbr             47234 non-null  float64
 6   calculatedfinishedsquarefeet  47234 non-null  int64  
 7   fips                          47234 non-null  int64  
 8   fullbathcnt                   47234 non-null  float64
 9   latitude                      47234 non-null  float64
 10  longitude                     47234 non-null  float64
 11  lotsizesquarefeet             47234 non-null  int64  
 12  propertycountylandusecode     47234 non-null  obje

In [64]:
df.calculatedbathnbr.value_counts(ascending = False)

2.0     20821
3.0      9576
1.0      7993
2.5      3722
4.0      1829
1.5       811
3.5       749
5.0       637
4.5       533
6.0       216
5.5       177
7.0        70
8.0        39
6.5        32
7.5        11
9.0         9
10.0        4
11.0        3
8.5         1
13.0        1
Name: calculatedbathnbr, dtype: int64

In [65]:
df.roomcnt.value_counts(ascending = False)

0.0     33488
6.0      3795
7.0      3471
8.0      2731
5.0      1749
9.0      1117
4.0       400
10.0      306
11.0       85
3.0        39
12.0       30
2.0         8
13.0        8
14.0        5
15.0        1
1.0         1
Name: roomcnt, dtype: int64

In [66]:
df.structuretaxvaluedollarcnt.value_counts(ascending = False), df.taxvaluedollarcnt.value_counts(ascending = False)

(100000.0    44
 200000.0    32
 32489.0     26
 95000.0     25
 90000.0     24
             ..
 243883.0     1
 317570.0     1
 70619.0      1
 522828.0     1
 154255.0     1
 Name: structuretaxvaluedollarcnt, Length: 34200, dtype: int64,
 455000.0    29
 600000.0    25
 500000.0    23
 450000.0    21
 585000.0    21
             ..
 153224.0     1
 156280.0     1
 256410.0     1
 119668.0     1
 409961.0     1
 Name: taxvaluedollarcnt, Length: 35608, dtype: int64)

In [ ]:
df.columns = df.columns.rename({'propertylandusetypeid': 'landuse_id',
                                'heatingorsystemtypeid': 'heat_id',
                                'id': 'transaction_id',
                                'bathroomcnt': 'bathrooms',
                                'bedroomcnt': 'bedrooms',
                                'calculatedbathnbr': 'bath_bed',
                                'fips': 'fips_id',
                                'fullbathcnt': 'full_bathrooms',
                                'lotsizesquarefeet': 'lot_size',
                                'propertylandusecode': 'property_landuse_code',
                                'regionidzip': 'zip_code',
                                'roomcnt': 'rooms',
                                'unitcnt': 'units',
                                'structuretaxvaluedollarcnt': 'structure_tax_value',
                                'taxvaluedollarcnt': 'tax_value',
                                'assessmentyear': 'year_assessed',
                                'landtaxvaluedollarcnt': 'land_tax_value',
                                'taxamount': 'tax_amount',
                                'logerror': 'log_error',
                                ''})